In [1]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

In [3]:
train = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 3360 files belonging to 6 classes.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.


2022-08-02 09:30:54.298391: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-02 09:30:54.298559: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
with tf.device('/cpu:0'):
    data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
        # layers.RandomContrast(0.2),
        # layers.RandomTranslation(height_factor=0.2,width_factor=0.3)
    ] 
    )

In [5]:
inputs = keras.Input(shape=(100, 100, 3))
x = inputs
# with tf.device('/cpu:0'):
#   x = data_augmentation(x)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.Conv2D(128, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.Conv2D(256, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(6, activation='softmax')(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.RMSprop(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 100, 100, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 conv2d_2 (Conv2D)           (None, 94, 94, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 47, 47, 32)       0         
 )                                                               
                                                             

In [6]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8
    ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/zenodo_01.h5",
        monitor="val_accuracy",
        save_best_only=True
    )
]

In [7]:
history = model.fit(
  train.cache(),
  epochs=50,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/50


2022-08-02 09:30:57.356954: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-02 09:30:57.357114: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.8008 - accuracy: 0.1568

2022-08-02 09:31:12.368026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 19s 133ms/step - loss: 1.8008 - accuracy: 0.1568 - val_loss: 1.7918 - val_accuracy: 0.1667
Epoch 2/50
105/105 [==============================] - 12s 114ms/step - loss: 1.8119 - accuracy: 0.1741 - val_loss: 1.7894 - val_accuracy: 0.2208
Epoch 3/50
105/105 [==============================] - 13s 128ms/step - loss: 1.6451 - accuracy: 0.3173 - val_loss: 1.3634 - val_accuracy: 0.4097
Epoch 4/50
105/105 [==============================] - 12s 118ms/step - loss: 1.1141 - accuracy: 0.5690 - val_loss: 0.9836 - val_accuracy: 0.6028
Epoch 5/50
105/105 [==============================] - 13s 120ms/step - loss: 0.8174 - accuracy: 0.6967 - val_loss: 0.9467 - val_accuracy: 0.6736
Epoch 6/50
105/105 [==============================] - 13s 125ms/step - loss: 0.6181 - accuracy: 0.7839 - val_loss: 2.6963 - val_accuracy: 0.3292
Epoch 7/50
105/105 [==============================] - 14s 135ms/step - loss: 0.4474 - accuracy: 0.8443 - val_loss: 0.6818 - val_accuracy: 0.7

In [8]:
model.evaluate(test)

23/23 [==============================] - 1s 38ms/step - loss: 0.5468 - accuracy: 0.9417


[0.5467965602874756, 0.9416666626930237]

In [9]:
# keras.models.save_model(model, "./models/zenodo_3.h5")

load_model = keras.models.load_model("models/zenodo_01.h5")
load_model.evaluate(test)

 1/23 [>.............................] - ETA: 5s - loss: 0.4645 - accuracy: 0.9375

2022-08-02 09:38:28.023601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 39ms/step - loss: 0.3098 - accuracy: 0.9486


[0.3097553253173828, 0.9486111402511597]